<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [1]:
from __future__ import print_function

%matplotlib inline
import numpy
import matplotlib.pyplot as plt

# Finite Volume Methods

## General Formulation of a Conservation Law

Define the value
$$
    Q^n_i \approx \frac{1}{\Delta x} \int^{x_{i+1/2}}_{x_{i-1/2}} q(x, t_n) dx \equiv \frac{1}{\Delta x} \int_{\mathcal{C}_i} q(x, t_n) dx
$$
where the $i$th **grid cell** is denoted by
$$
    \mathcal{C}_i \equiv (x_{i-1/2}, x_{i+1/2})
$$
and $\Delta x = x_{i+1/2} - x_{i-1/2}$.  

If $q(x,t)$ is smooth then the average value 
$$
    Q^n_i = q(x_i, t_n) + \mathcal{O}(\Delta x^2)
$$

With this definition we now return to our version of the conservation  law written over the cell $\mathcal{C}_i$:
$$
    \frac{\text{d}}{\text{d} t} \int_{\mathcal{C}_i} q(x,t) dx = f(q(x_{i-1/2}, t)) - f(q(x_{i+1/2}, t)).
$$
Now take the following steps:
$$\begin{aligned}
    \frac{\text{d}}{\text{d} t} \int_{\mathcal{C}_i} q(x,t) dx &= f(q(x_{i-1/2}, t)) - f(q(x_{i+1/2}, t)) & & \text{Original}\\
    \int_{\mathcal{C}_i} q(x,t_{n+1}) dx - \int_{\mathcal{C}_i} q(x,t_{n}) dx &= \int^{t_{n+1}}_{t_n} \left[ f(q(x_{i-1/2}, t)) - f(q(x_{i+1/2}, t)) \right ] dt & & \text{Integrate over } [t_n, t_{n+1}] \\
    \frac{1}{\Delta x} \int_{\mathcal{C}_i} q(x,t_{n+1}) dx &= \frac{1}{\Delta x} \int_{\mathcal{C}_i} q(x,t_{n}) dx + \frac{1}{\Delta x} \int^{t_{n+1}}_{t_n} \left[ f(q(x_{i-1/2}, t)) - f(q(x_{i+1/2}, t)) \right ] dt & & \text{Rearrange and divide by } \Delta x \\
    Q^{n+1}_i &= Q^n_i + \frac{1}{\Delta x} \int^{t_{n+1}}_{t_n} \left[ f(q(x_{i-1/2}, t)) - f(q(x_{i+1/2}, t)) \right ] dt & & \text{Approximate with averages } Q^{n+1,n}_i \\
    Q^{n+1}_i &= Q^n_i - \frac{\Delta t}{\Delta x} (F^n_{i+1/2} - F^n_{i-1/2}) & & \text{Replace with }F_{i\pm1/2} \\
\end{aligned}$$
where
$$
    F^n_{i\pm 1/2} \approx \frac{1}{\Delta t} \int^{t_{n+1}}_{t_n} f(q(x_{i\pm 1/2}, t)) dt.
$$

This approximate flux should have a formula akin to
$$
    F^n_{i-1/2} = \mathcal{F}(Q^n_{i-1}, Q^n_i)
$$
where here we are thinking of $\mathcal{F}$ as a numerical flux function.  The question then turns to how to prescribe a **numerical flux function**.  Note that this implies that the update to $Q^{n+1}_i$ depends on three previous values, $Q^n_{i-1}, Q^n_{i}, Q^n_{i+1}$.

We can also now discuss how conservation should be described numerically.  If
$$
    \Delta x \sum^J_{i=I} Q^{n+1}_i = \Delta x \sum^J_{i=I} Q^n_i - \frac{\Delta t}{\Delta x} (F^n_{J+1/2} - F^n_{I-1/2})
$$
and the sum of the fluxes cancels then **global conservation** is maintained up to the boundaries of the domain.

We can also rewrite the numerical methods as
$$
    \frac{Q^{n+1}_i - Q^n_i}{\Delta t} + \frac{F^n_{i+1/2} - F^n_{i-1/2}}{\Delta x} = 0
$$
which implies that we have formulated a first order method in terms of finite difference methods.  We will find that this is not entirely the case depending on what is prescribed for $F_{i\pm 1/2}$.

## Numerical Flux for the Diffusion Equation

An interesting exercise at this point is to consider a more general method to the above approach.  Consider again the parabolic heat equation.  We can define a flux function that contains a derivative of $q$ such that
$$
    f(q_x, x) = -\beta(x) q_x.
$$
Cleary this gives us the heat equation if we substitute this flux function into our general equations
$$
    q_t + f(q_x, x)_x = q_t - (\beta(x) q_x)_x = 0.
$$

One way to prescribe a numerical flux would be the following:
$$
    \mathcal{F}(Q_{i-1}, Q_i) = -\beta_{i-1/2} \left ( \frac{Q_i - Q_{i-1}}{\Delta x} \right ).
$$
If we use this in our previous definition we then have
$$
    Q^{n+1}_i = Q^n_i + \frac{\Delta t}{\Delta x^2} \left[ \beta_{i+1/2}(Q^n_{i+1} - Q^n_i) - \beta_{i-1/2} (Q^n_i - Q^n_{i-1}) \right]
$$
noting that if $\beta$ is constant we have
$$
    Q^{n+1}_i = Q^n_i + \beta \frac{\Delta t }{\Delta x^2} \left[ Q^n_{i+1} - 2 Q^n_i + Q^n_{i-1} \right]
$$

We however know that the previous scheme is not ideal.  Instead if we use the well-known Crank-Nicolson scheme we have
$$
    Q^{n+1}_i = Q^n_i + \frac{\Delta t}{2 \Delta x^2} \left[ \beta_{i+1/2}(Q^n_{i+1} - Q^n_i) - \beta_{i-1/2} (Q^n_i - Q^n_{i-1}) + \beta_{i+1/2}(Q^{n+1}_{i+1} - Q^{n+1}_i) - \beta_{i-1/2} (Q^{n+1}_i - Q^{n+1}_{i-1})\right ]
$$
with the flux
$$
    F^n_{i-1/2} = -\frac{1}{\Delta x} \left[ \beta_{i-1/2} (Q^n_i - Q^n_{i-1}) + \beta_{i-1/2} (Q^{n+1}_i - Q^{n+1}_{i-1}) \right ]
$$

## Convergence

For any numerical method we desire that as $\Delta x, \Delta t \rightarrow 0$ that the numerical solution converges to the true solution.  This generally requires the following conditions:

1. The method must be consistent:  the approximation is valid locally.
1. The method must be stable:  small errors do not accumalate too fast.

### Consistency

In the case of numerical fluxes we want to require that the numerical flux reduces to the true flux in some sense.  One way to require this is to ensure for regions where $q$ is constant that the numerical flux agrees with the flux function:
$$
    \mathcal{F}(q, q) = f(q).
$$
A more formal definition is to ensure that there is some sort of Lipschitz continuity of the form
$$
    |\mathcal{F}(Q_{i-1}, Q_i) - f(q)| \leq L \text{max}(|Q_i - q|, |Q_{i-1} - q|).
$$

### Stability

Stability can take on many different forms, many of which will be discussed later.  Here we will consider the necessary condition of CFL stability.  This is usually expressed as

$$
    \nu \equiv \left |\frac{u \Delta t}{\Delta x} \right | \leq 1.
$$

## Numerical Fluxes

We now will consider a number of different flux defintions and consider their viability.

### Example: Unstable Flux

Consider the flux
$$
    F^n_{i-1/2} = \mathcal{F}(Q^n_{i-1}, Q^n_i) = \frac{1}{2} [f(Q^n_{i-1}) + f(Q^n_i)]
$$
leading to the method
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{2 \Delta x} [f(Q^n_{i+1}) + f(Q^n_{i-1})].
$$

Unfortunately this method is unstable!

### Example: Lax-Friedrichs Method

The classical Lax-Friedrichs method has the flux function
$$
    \mathcal{F}(Q^n_{i-1}, Q^n_i) = \frac{1}{2} [f(Q^n_{i-1}) + f(Q^n_i)] - \frac{\Delta x}{2 \Delta t} (Q^n_i - Q^n_{i-1})
$$
and the full method
$$
    Q^{n+1}_i = \frac{1}{2} (Q^n_{i-1} + Q^n_{i+1}) - \frac{\Delta t}{2 \Delta x} [ f(Q^n_{i+1}) - f(Q^n_{i-1})]
$$
leading to a first order accurate method.

### Example: Upwind Methods

We know from finite difference methods that upwind methods have a significant advantage over more general methods by simply looking at where the flow is coming from thereby satisfying the CFL condition.  In terms of fluxes the upwind method for advection is
$$
    F^n_{i-1/2} = u Q^n_{i-1}
$$
if $u \ge 0$.  Using this flux function gives us the standard upwind method
$$
    Q^{n+1}_i = Q^n_i - \frac{u \Delta t}{\Delta x} (Q^n_i - Q^n_{i-1}).
$$

Note that the last equation has a difference of the values of $Q$ meaning that we can write this in terms of our waves.  In this context the numerical method can be written as
$$
    Q^{n+1}_i = Q^n_i - \frac{u \Delta t}{\Delta x} \mathcal{W}_{i-1/2}.
$$

We can also write the method in terms of $u \leq 0$ as
$$
    Q^{n+1}_i = Q^n_i - \frac{u \Delta t}{\Delta x} \mathcal{W}_{i+1/2}.
$$

We can generalize this so that we can define a flux function
$$
    F^n_{i-1/2} = u^- Q^n_i + u^+ Q^n_{i-1}
$$
where
$$
    u^+ = \text{max}(u, 0) \text{   and    } u^- = \text{min}(u, 0)
$$
allowing us to write a more general method as
$$
    Q^{n+1}_i = Q^n_i - \frac{\Delta t}{\Delta x} (u^+ \mathcal{W}_{i-1/2} + u^- \mathcal{W}_{i+1/2})
$$

## Godunov's Method

### REA Algorithm

1. 

### Numerical Flux for Godunov's Method

### Wave-Propagation Form of Godunov's Method

### Roe's Method